# 2. 2D example

In [ ]:
import numpy as np
import torch

In [ ]:
# set logger and enforce reproducibility
from GPErks.log.logger import get_logger
from GPErks.utils.random import set_seed
log = get_logger()
seed = 8
set_seed(seed)  # reproducible sampling


<br/>

**2D function example**: Currin et al. (1988)

$f(x) = \left[1 - \exp{\left(1 - \dfrac{1}{2 x_2}\right)}\right]\,\left(\dfrac{2300 x_{1}^3 + 1900 x_{1}^2 + 2092 x_{1} + 60}{100 x_{1}^3 + 500 x_{1}^2 + 4 x_{1} + 20}\right)$

<br/>


In [ ]:
# function to learn (normally a high-dimensional, expensive deterministic model)
from GPErks.utils.test_functions import currin_exp
f = lambda X: np.array([currin_exp(x) for x in X])
D = 2

In [ ]:
# build dataset
from GPErks.gp.data.dataset import Dataset
dataset = Dataset.build_from_function(
    f,
    D,
    n_train_samples=20,
    n_test_samples=25,
    design="lhs",
    seed=seed,
)
dataset.plot()

In [ ]:
dataset.plot_pairwise()

In [ ]:
# choose likelihood
from gpytorch.likelihoods import GaussianLikelihood
likelihood = GaussianLikelihood()

In [ ]:
# choose mean function
from gpytorch.means import LinearMean
mean_function = LinearMean(input_size=dataset.input_size)

In [ ]:
# choose kernel
from gpytorch.kernels import RBFKernel, ScaleKernel
kernel = ScaleKernel(RBFKernel(ard_num_dims=dataset.input_size))

In [ ]:
# choose metrics
from torchmetrics import MeanSquaredError, R2Score
metrics = [MeanSquaredError(), R2Score()]

In [ ]:
# define experiment
from GPErks.gp.experiment import GPExperiment
experiment = GPExperiment(
    dataset,
    likelihood,
    mean_function,
    kernel,
    n_restarts=3,
    metrics=metrics,
    seed=seed  # reproducible training
)

In [ ]:
# choose training options: device + optimizer
device = "cpu"
optimizer = torch.optim.Adam(experiment.model.parameters(), lr=0.1)

In [ ]:
# train model
from GPErks.train.emulator import GPEmulator
emulator = GPEmulator(experiment, device)
emulator.train(optimizer)

In [ ]:
# inference on stored test set
from GPErks.perks.inference import Inference
inference = Inference(emulator)
inference.summary()
inference.plot()

In [ ]:
# bonus: inference on 2-dimensional grid
inference.interpolate_2Dgrid()  # can add function f as optional argument

In [ ]:
# perk n.2: diagnostics (L.S. Bastos and A. O’Hagan (2009) doi:10.1198/TECH.2009.08019)
from GPErks.perks.diagnostics import Diagnostics
diagnostics = Diagnostics(emulator)

y_mean, y_std, y_covar = emulator.predict(dataset.X_test, with_covar=True)
print( y_covar.shape )

In [ ]:
import matplotlib.pyplot as plt
fig, axis = plt.subplots(1, 1)
h = axis.imshow(y_covar)
cbar = fig.colorbar(h, ax=axis)
fig.tight_layout()
plt.show()

In [ ]:
diagnostics.summary()

In [ ]:
diagnostics.plot(uncorrelated=False)

In [ ]:
1 - 2/25